

## ДЗ по лекции №7 "Классификация в NLP": Сделать классификацию данных fakenews
Используя ноутбук занятия (также размещен в папке Materials) и данные fakenews, 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.


In [1]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2022-07-10 16:24:08--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.04s   

2022-07-10 16:24:08 (31.2 MB/s) - ‘Constraint_Train.csv’ saved [1253562/1253562]



In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('Constraint_Train.csv')

In [4]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [5]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:01<00:00, 3413.30it/s]


In [8]:
from gensim.models.word2vec import Word2Vec
%time model_tweets = Word2Vec(sentences, workers=4, size=300, min_count=3, window=5, iter=15)

CPU times: user 9.11 s, sys: 99.3 ms, total: 9.21 s
Wall time: 5.37 s


In [9]:
model_tweets.wv.most_similar('france')

[('front', 0.9463140964508057),
 ('aires', 0.9376326203346252),
 ('floor', 0.9345133304595947),
 ('protesting', 0.9301860928535461),
 ('tower', 0.9299658536911011),
 ('crying', 0.9280542731285095),
 ('deceased', 0.9249671697616577),
 ('parliament', 0.9245487451553345),
 ('buenos', 0.9210494756698608),
 ('suggesting', 0.9208232164382935)]

In [10]:
model_tweets.wv.init_sims()

In [11]:
import numpy as np

In [12]:
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [13]:
features = [get_text_embedding(text) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:02<00:00, 2430.20it/s]


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)

In [16]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=5000)

In [17]:
from sklearn.metrics import classification_report

In [18]:
predicted = model.predict(X_test)

In [19]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.93      0.92      1000
        real       0.94      0.92      0.93      1119

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



###  Вариант 1. Что будет, если использовать самый наивный метод (CountVectorizer)?

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
vec = CountVectorizer()

In [22]:
bow = vec.fit_transform(df.tweet)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(bow, df.label, test_size=0.33)
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=5000)

In [24]:
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.92      0.92       984
        real       0.93      0.93      0.93      1135

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



Мы видим, что метрика f1-score несущественно ухудшилась (с 0.93 до 0.92).

## Вариант 2. Попробуем векторизатор Tf-Idf с параметрами по умолчанию (разбивка только на униграммы).

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
tfidf = TfidfVectorizer()

In [27]:
bow = tfidf.fit_transform(df.tweet)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(bow, df.label, test_size=0.33)
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=5000)

In [29]:
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.91      0.91      0.91      1019
        real       0.92      0.92      0.92      1100

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



Мы видим, что метрика f1-score осталась прежней (0.92). 

## Вариант 3. Теперь попробуем векторизатор Tf-Idf с разбивкой твитов на n-граммы от 1 (униграмм) до 3 (триграмм).

In [30]:
tfidf_ngrams = TfidfVectorizer(ngram_range = (1, 3))

In [31]:
bow = tfidf_ngrams.fit_transform(df.tweet)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(bow, df.label, test_size=0.33)
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=5000)

In [33]:
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.89      0.92      0.90      1003
        real       0.92      0.90      0.91      1116

    accuracy                           0.91      2119
   macro avg       0.91      0.91      0.91      2119
weighted avg       0.91      0.91      0.91      2119



На N-граммах от 1 до 3 метрика f1-score еще незначительно ухудшилась (до 0.91). 

Можно сделать вывод, что использование TF-Idf-векторизатора вместо самого простого CountVectorizer в данном случае не привело к повышению точности предсказаний. При этом CountVectorizer работает быстрее, чем TF-IDF (на большом датасете было бы заметно).

### Часть 2. PyTorch + LSTM

In [34]:
labels = (df.label == 'real').astype(int).to_list()

Нужно заранее задать размер для максимальной длины предложений.

In [35]:
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [36]:
max_len

1592

Это слишком много. Но какая длина обычно?

In [37]:
from collections import Counter
fd = Counter([len(tokens) for tokens in token_lists])

In [38]:
fd.most_common(10)

[(20, 178),
 (25, 174),
 (22, 170),
 (18, 170),
 (19, 168),
 (21, 168),
 (16, 163),
 (17, 162),
 (15, 160),
 (23, 156)]

## Вариант 1. Зададим максимальную длину предложений в 200 символов.

Возьмём те же w2v эмбеддинги.

In [39]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

In [40]:
features = [get_word_embedding(text, 200) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:03<00:00, 2055.71it/s]


In [41]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim

In [43]:
len(features[0][0])

300

In [44]:
len(X_train)

4301

In [45]:
len(X_train[0])

200

In [46]:
len(X_train[0][0])

300

In [47]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [48]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  """Entry point for launching an IPython kernel.


In [49]:
in_data.shape

torch.Size([4301, 200, 300])

In [50]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [51]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [52]:
train_one_epoch(in_data, targets)

100%|██████████| 269/269 [03:17<00:00,  1.36it/s]

tensor(0.6907, grad_fn=<BinaryCrossEntropyBackward0>)


Что получилось?

In [53]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [54]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [55]:
result = (output > 0.5) == targets_test

In [56]:
result.sum().item() / len(result)

0.5219443133553563

## Вариант 2. Попробуем взять максимальную длину предложений 300 (вместо 200).

In [57]:
features = [get_word_embedding(text, 300) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:04<00:00, 1549.66it/s]


In [58]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [59]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [60]:
in_data.shape

torch.Size([4301, 300, 300])

In [61]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [62]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [63]:
train_one_epoch(in_data, targets)

100%|██████████| 269/269 [02:29<00:00,  1.81it/s]

tensor(0.6910, grad_fn=<BinaryCrossEntropyBackward0>)


In [64]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [65]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [66]:
result = (output > 0.5) == targets_test

In [67]:
result.sum().item() / len(result)

0.5403492213308164

Видим, что результат улучшился с 0.52 до 0.54 при использовании макс.длины предложений 300 вместо 200, но в Гугл Колаб доступная память была практически полностью использована (риск аварийного завершения среды исполнения).

## Вариант 3. Попробуем использовать другой оптимизатор (Adam  вместо SGD) и другую функцию потерь ("Mean Squared Error" вместо "Binary Cross Entropy").

In [68]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [69]:
train_one_epoch(in_data, targets)

100%|██████████| 269/269 [03:19<00:00,  1.35it/s]

tensor(0.2491, grad_fn=<MseLossBackward0>)


In [70]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [71]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [72]:
result = (output > 0.5) == targets_test

In [73]:
result.sum().item() / len(result)

0.5403492213308164

Мы видим, что обучение прошло чуть дольше, результат при этом остался прежним (0.54).

## Вариант 4. Попробуем уменьшить Learning Rate с 0.01 до 0.001. Остальные параметры модели оставим, как в варианте №3.

In [74]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [75]:
train_one_epoch(in_data, targets)

100%|██████████| 269/269 [02:33<00:00,  1.75it/s]

tensor(0.2489, grad_fn=<MseLossBackward0>)


In [76]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [77]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [78]:
result = (output > 0.5) == targets_test

In [79]:
result.sum().item() / len(result)

0.5403492213308164

Обучение прошло чуть быстрее, но результат остался прежним (0.54).